In [1]:
%pip install pandas

import gc
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
input_parquet_file_path = '../data/interm/1.0_taxdata.parquet'
output_parquet_file_path = '../data/interm/2.0_taxdata.parquet'

input_department_names_file_path = '../data/department_names.csv'

In [3]:
df = pd.read_parquet(input_parquet_file_path)

departments_df = pd.read_csv(input_department_names_file_path)

# Correct, clean and fill the commune identifiers data

In [4]:
# Strip any leading "- " from 'Libellé commune' and trim extra whitespace
df['Libellé commune'] = df['Libellé commune'].map(lambda x: x.lstrip('- ').strip() if isinstance(x, str) else x)

# Clean the 'DEPARTEMENT' field
df['DEPARTEMENT'] = df['DEPARTEMENT'].astype(str).str.strip().replace('\n', '', regex=True)
df['DEPARTEMENT'] = df['DEPARTEMENT'].fillna('').map(lambda x: x.strip() if pd.notna(x) else '')
df['DEPARTEMENT'] = df['DEPARTEMENT'].apply(lambda x: '0' + x if len(x) == 1 else x)

# Clean the 'COMMUNE' field
df['COMMUNE'] = df['COMMUNE'].where(df['COMMUNE'] != 999, 56)
df['COMMUNE'] = df['COMMUNE'].fillna(0).astype(int)

# Clean the DEPARTEMENT field
# Example: 96 - 101. Should be 971 - 01
# Example: 97 - 201. Should be 972 - 01
# Example: 98 - 301. Should be 973 - 01
# Example: 99 - 401. Should be 974 - 01
df['DEPARTEMENT'] = df['DEPARTEMENT'].where(df['DEPARTEMENT'] != '96', '971')
df['DEPARTEMENT'] = df['DEPARTEMENT'].where(df['DEPARTEMENT'] != '97', '972')
df['DEPARTEMENT'] = df['DEPARTEMENT'].where(df['DEPARTEMENT'] != '98', '973')
df['DEPARTEMENT'] = df['DEPARTEMENT'].where(df['DEPARTEMENT'] != '99', '974')

# Convert COMMUNE column to strings for transformation
df['COMMUNE'] = df['COMMUNE'].astype(str)
df['COMMUNE'] = df.apply(lambda row: str(row['COMMUNE'][1:]) if row['DEPARTEMENT'] in ['971', '972','973', '974'] and len(row['COMMUNE']) == 3 else row['COMMUNE'], axis=1)
df['COMMUNE'] = df['COMMUNE'].astype(int)

# Save the results

In [5]:
# After cleaning up the DEPARTMENT codes, add a new column DEPARTMENT_LABEL to have all the names of the departments
merged_df = pd.merge(df, departments_df[['DEPARTEMENT', 'DEPARTEMENT_LABEL']], on='DEPARTEMENT', how='left')

# Save the processed data back to a Parquet file
merged_df.to_parquet(output_parquet_file_path)

del df
del departments_df
del merged_df
gc.collect()

0